In [30]:
import numpy as np

In [31]:
def grad_phi(l, w, B):
    
    size = len(w)
    
#     print(np.dot(B, np.ones(size)).A1)
    grad_phi_1 = - l + np.dot(B, np.ones(size)).A1 / np.dot(np.dot(np.ones(size).T, B), np.ones(size))
    grad_phi_2 = - w + np.dot(B.T, np.ones(size)).A1 / np.dot(np.dot(np.ones(size).T, B), np.ones(size))
    
    return grad_phi_1, grad_phi_2, np.concatenate((grad_phi_1.A1, grad_phi_2.A1))

In [32]:
def phi(x, l, w, B):
    print(len(x), len(w))
    size = len(w)
    
    return - np.dot(x[:size], l) - np.dot(x[size:], w) + np.log(np.dot(np.dot(np.ones(size).T, B), np.ones(size)).A1[0])

In [33]:
# x = np.zeros(2 * size)
# print(x)
# B = np.exp(-beta * T - x[:size] - x[size:])
# print(B)
# phi(x, l, w, B)

In [34]:
# x = 150 *np.ones(2 * size)
# print(x)
# B = np.exp(-beta * T - x[:size] - x[size:])
# # print(B)
# phi(x, l, w, B)

In [35]:
# x = np.array(range(2 * size))
# print(x)
# B = np.exp(-beta * T - x[:size] - x[size:])
# # print(B)
# phi(x, l, w, B)

In [36]:
np.dot(np.ones(2), np.matrix([[2,3], [4,5]]))

matrix([[6., 8.]])

In [37]:
# np.dot(np.ones(size).T, B)

In [38]:
def f(correspondance_matrix, cost_matrix, gamma):
    
    return np.sum(correspondance_matrix * np.log(correspondance_matrix)) + gamma * np.sum(corresponance_matrix * cost_matrix)

In [39]:
import pandas as pd

In [40]:
df = pd.read_csv('moscow.csv')

In [41]:
df.head()

,area_i,area_j,live_i_work_j,time_ij,dist_ij
0,1,1,40,26,2.2
1,1,2,4,49,8.1
2,1,3,3,35,6.8
3,1,4,2,75,9.3
4,1,5,2,30,8.0


In [42]:
size = df.area_i.max()

In [43]:
df.area_i = df.area_i.astype(int)
df.area_j = df.area_j.astype(int)

In [44]:
times_matrix, distance_matrix, corresponance_matrix = np.zeros((size, size)), np.zeros((size, size)), np.zeros((size, size))

In [45]:
for i in df.index:
    
    area_i, area_j, live_work_ij, time_ij, dist_ij = df.loc[i]
    area_i, area_j = int(area_i) - 1, int(area_j) - 1
    
    times_matrix[area_i, area_j] = time_ij
    distance_matrix[area_i, area_j] = dist_ij
    corresponance_matrix[area_i, area_j] = live_work_ij

In [62]:

# size = 10
beta = 1

# T = np.matrix([np.random.uniform(size=size) for _ in range(size)])
# T /= T.sum(axis=1)

T = 2 * np.matrix(times_matrix ** 4)
T /= T.sum(axis=1)

# l, w = np.random.uniform(0, 1, size), np.random.uniform(0, 1, size)  
l, w = df.groupby('area_i')['live_i_work_j'].sum(), df.groupby('area_j')['live_i_work_j'].sum()
l /= l.sum()
w /= w.sum()

lambdas = np.ones(2 * size)
#np.concatenate((np.zeros(size), np.ones(size))) 


In [63]:
eps_f = 0.001

In [64]:
k = 0
while True:
    
    lambdas_l, lambdas_w = lambdas[:size].copy(), lambdas[size:].copy()
    B = np.exp(-beta * T + lambdas_l + lambdas_w)
    
    if k % 2 == 0:
        lambdas_l_new = (lambdas_l + np.log(l) - np.log(np.dot(B, np.ones(size))).A1).values
        lambdas_w_new = lambdas_w.copy() 

    else:
        lambdas_l_new = lambdas_l.copy()
        lambdas_w_new = (lambdas_w + np.log(w) - np.log(np.dot(B.T, np.ones(size))).A1).values

    k = k + 1 
    lambdas_new = np.concatenate((lambdas_l_new, lambdas_w_new))
    
    if np.linalg.norm(lambdas_new - lambdas) / np.linalg.norm(lambdas) < eps_f:
        print(np.linalg.norm(lambdas_new - lambdas) / np.linalg.norm(lambdas))
        print('iter =', k)
        break
    
    lambdas = lambdas_new.copy()




0.000999788926135928
iter = 1414


In [49]:
L = 1
L_new = None
a = 0
# size = 10
beta = 1

# T = np.matrix([np.random.uniform(size=size) for _ in range(size)])
# T /= T.sum(axis=1)

T = 2 * np.matrix(times_matrix ** 4)
T /= T.sum(axis=1)

# l, w = np.random.uniform(0, 1, size), np.random.uniform(0, 1, size)  
l, w = df.groupby('area_i')['live_i_work_j'].sum(), df.groupby('area_j')['live_i_work_j'].sum()
l /= l.sum()
w /= w.sum()

# lambdas_l, lambdas_w = np.zeros(size), np.zeros(size)
#x_new = np.random.uniform(size=2 * size)#np.concatenate((np.zeros(size), np.ones(size))) 
#y = np.random.uniform(size=2 * size)#np.concatenate((np.zeros(size), np.ones(size))) 
#v_new = np.random.uniform(size=2 * size)#np.concatenate((np.zeros(size), np.ones(size)))
x_new = np.ones(2 * size)#np.concatenate((np.zeros(size), np.ones(size))) 
y = np.ones(2 * size)#np.concatenate((np.zeros(size), np.ones(size))) 
v_new = np.ones(2 * size)#np.concatenate((np.zeros(size), np.ones(size)))
# x, y, v = 15* np.ones(2 * size), np.ones(2 * size), 15 *np.ones(2 * size)

d_hat = 1

In [50]:
while True:
    if L_new is not None:
        L = L_new
        
    L_new = L / 2
    
    x = x_new.copy()
    v = v_new.copy()
    while True:
    
        a_new = 1 / (2 * L_new) + np.sqrt(1 / (4 * L_new ** 2) + a ** 2 * L / L_new)
        tau = 1 / (a_new * L_new)
        
        y = tau * v + (1 - tau) * x
        print(y)
        lambdas_l, lambdas_w = y[:size].copy(), y[size:].copy()

        B = np.exp(-beta * T + lambdas_l + lambdas_w)
#         print(B)
        grad_phi_1, grad_phi_2, grad_phi_value = grad_phi(y[:size], y[size:], B)
        
        if np.linalg.norm(grad_phi_1) >= np.linalg.norm(grad_phi_2):
            lambdas_l_new = (lambdas_l + np.log(l) - np.log(np.dot(B, np.ones(size))).A1).values
            lambdas_w_new = lambdas_w.copy() 
            print(1)
        else:
            lambdas_l_new = lambdas_l.copy()
            lambdas_w_new = (lambdas_w + np.log(w) - np.log(np.dot(B.T, np.ones(size))).A1).values
            print(2)
        x_new = np.concatenate((lambdas_l_new, lambdas_w_new))
   
        v_new = v - a_new * grad_phi_value
        
        B_x_new = np.exp(-beta * T + x_new[:size] + x_new[size:])
        B_x = np.exp(-beta * T + x[:size] + x[size:])
        
        print(phi(x_new, l, w, B_x_new), phi(x, l, w, B_x), phi(y, l, w, B))
#         print(phi(x_new, l, w, B_x_new) - phi(y, l, w, B) - np.linalg.norm(grad_phi_value) ** 2 / (2 * L_new))

        if phi(x_new, l, w, B_x_new) <= phi(y, l, w, B) - np.linalg.norm(grad_phi_value) ** 2 / (2 * L_new):
            d = B / np.dot(np.dot(np.ones(size).T, B), np.ones(size))
#             print(np.dot(np.dot(np.ones(size).T, B), np.ones(size)))
            d_hat_new = (a_new * d + L * a ** 2 * d_hat) / (L_new * a_new ** 2)
            break
        
        L_new *= 2
        print('L_new', L_new)
        
        
    print(abs(f(d_hat_new, T, beta) + phi(x_new, l, w, B_x_new)))
    print(np.linalg.norm(np.dot(d_hat_new.T, np.ones(size).T).A1 - l), np.linalg.norm(np.dot(d_hat_new, np.ones(size).T).A1 - w))
    
    if abs(f(d_hat_new, T, beta) + phi(x_new, l, w, B)) < eps_f:
        break
    
    d_hat = d_hat_new.copy()


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
2
44 22
44 22
44 22
5.84454260846877 6.142907172373567 6.142907172373567
44 22
44 22
L_new 1.0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
2
44 22
44 22
44 22
5.84454260846877 6.142907172373567 6.142907172373567
44 22
44 22
L_new 2.0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
2
44 22
44 22
44 22
5.84454260846877 6.142907172373567 6.142907172373567
44 22
44 22
L_new 4.0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
2
44 22
44 22
44 22
5.84454260846877 6.142907172373567 6.142907172373567
44 22
44 22
L_new 8.0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 

44 22
44 22
44 22
5.844541903153971 5.844541932019876 6.14273218296954
44 22
44 22
44 22
1834.7008084790143
0.1287572603082837 0.17176217597555368
44 22
[1.2324132  1.23244214 1.23238916 1.23242465 1.23230631 1.23242126
 1.2324299  1.23243366 1.23241934 1.23234646 1.23242467 1.23244665
 1.23241019 1.23242076 1.23243127 1.23239314 1.23239739 1.23239864
 1.23240108 1.23228869 1.23231797 1.23235871 1.23276909 1.23274063
 1.23250831 1.23255608 1.23245158 1.23248819 1.23277102 1.23284039
 1.23260905 1.2321815  1.23204921 1.23253408 1.23229228 1.23292377
 1.23283154 1.23197549 1.23195651 1.23203312 1.23227225 1.23195679
 1.23201299 1.23196138]
2
44 22
44 22
44 22
5.844541874061339 5.844541903153971 6.1427250061195515
44 22
44 22
L_new 128.0
[1.2324132  1.23244214 1.23238916 1.23242465 1.23230631 1.23242126
 1.2324299  1.23243366 1.23241934 1.23234646 1.23242467 1.23244665
 1.23241019 1.23242076 1.23243127 1.23239314 1.23239739 1.23239864
 1.23240108 1.23228869 1.23231797 1.23235871 1.2327690

44 22
44 22
44 22
1834.7010035469973
0.12871848899317992 0.17176203132892437
44 22
[1.42704811 1.42710132 1.427004   1.42706917 1.42685163 1.42706299
 1.42707883 1.42708574 1.42705943 1.42692552 1.42706919 1.42710959
 1.42704263 1.427062   1.42708128 1.42701129 1.4270191  1.42702138
 1.42702587 1.42681938 1.42687314 1.42694803 1.42769938 1.42764707
 1.42722211 1.4273093  1.42711882 1.42718505 1.4277028  1.4278298
 1.42740632 1.42662419 1.42638151 1.42726887 1.4268265  1.42798256
 1.42781362 1.42624683 1.42621206 1.42635226 1.42678991 1.42621337
 1.426316   1.42622126]
2
44 22
44 22
44 22
5.84454125842636 5.844541275437836 6.142573815072669
44 22
44 22
L_new 256.0
[1.42704811 1.42710132 1.427004   1.42706917 1.42685163 1.42706299
 1.42707883 1.42708574 1.42705943 1.42692552 1.42706919 1.42710959
 1.42704263 1.427062   1.42708128 1.42701129 1.4270191  1.42702138
 1.42702587 1.42681938 1.42687314 1.42694803 1.42769938 1.42764707
 1.42722211 1.4273093  1.42711882 1.42718505 1.4277028  1.42

0.12869342158379438 0.17176193766657305
44 22
[1.55666455 1.55673392 1.5566071  1.556692   1.55640836 1.55668398
 1.55670461 1.55671362 1.55667931 1.55650477 1.55669203 1.55674469
 1.55665743 1.55668265 1.55670776 1.55661659 1.55662675 1.55662972
 1.55663557 1.55636642 1.55643649 1.55653412 1.55751136 1.55744316
 1.55689073 1.55700394 1.55675687 1.55684236 1.55751572 1.55768089
 1.55713016 1.55611341 1.5557974  1.55695124 1.55637614 1.55787967
 1.55765985 1.55562246 1.55557722 1.55575951 1.5563286  1.55557954
 1.55571282 1.5555894 ]
2
44 22
44 22
44 22
5.844540848181675 5.844540866797937 6.142473731021894
44 22
44 22
L_new 256.0
[1.55666455 1.55673392 1.5566071  1.556692   1.55640836 1.55668398
 1.55670461 1.55671362 1.55667931 1.55650477 1.55669203 1.55674469
 1.55665743 1.55668265 1.55670776 1.55661659 1.55662675 1.55662972
 1.55663557 1.55636642 1.55643649 1.55653412 1.55751136 1.55744316
 1.55689073 1.55700394 1.55675687 1.55684236 1.55751572 1.55768089
 1.55713016 1.55611341 1.555

44 22
5.8445405600019145 5.844540579745463 6.1424037122806805
44 22
44 22
L_new 256.0
[1.64767891 1.64775963 1.64761209 1.64771085 1.64738078 1.64770154
 1.64772552 1.64773601 1.64769609 1.64749302 1.64771087 1.64777215
 1.64767065 1.64769995 1.64772916 1.64762313 1.64763494 1.64763838
 1.64764521 1.64733206 1.64741356 1.64752718 1.64866252 1.64858317
 1.64794159 1.64807296 1.64778643 1.64788525 1.64866752 1.64885941
 1.64821961 1.64703877 1.64667134 1.64801164 1.64734369 1.64909041
 1.64883497 1.64646827 1.64641571 1.64662744 1.6472885  1.64641889
 1.64657357 1.64643003]
2
44 22
44 22
44 22
5.8445405600019145 5.844540579745463 6.1424037122806805
44 22
44 22
44 22
1834.7012216917544
0.1286746817725635 0.17176186757603673
44 22
[1.65393787 1.65401937 1.65387041 1.65397012 1.65363686 1.65396072
 1.65398494 1.65399552 1.65395522 1.65375018 1.65397014 1.65403201
 1.65392953 1.65395912 1.6539886  1.65388155 1.65389348 1.65389695
 1.65390384 1.65358767 1.65366996 1.65378467 1.65493087 1.6548

...

44 22
44 22
5.844528360744867 5.84452836497684 6.139568291957669
44 22
44 22
44 22
1834.7044249419598
0.1279627131478263 0.17175916395425284
44 22
[5.48623618 5.48679706 5.48577958 5.48645809 5.48416302 5.48639805
 5.48656126 5.48663396 5.48635695 5.48495119 5.48645727 5.48688239
 5.48618274 5.48638068 5.48658085 5.48585374 5.48593428 5.48595645
 5.48600539 5.48383792 5.48439973 5.48518922 5.4927774  5.49222669
 5.48797675 5.48882918 5.48700017 5.48757802 5.49279923 5.49408036
 5.48981252 5.48198658 5.47948293 5.4884042  5.48397278 5.49563538
 5.49391868 5.47815436 5.47780156 5.47920969 5.48361143 5.47790261
 5.47891024 5.47792524]
2
44 22
44 22
44 22
5.84452835651186 5.844528360744867 6.139567337772284
44 22
44 22
L_new 4096.0
[5.48623618 5.48679706 5.48577958 5.48645809 5.48416302 5.48639805
 5.48656126 5.48663396 5.48635695 5.48495119 5.48645727 5.48688239
 5.48618274 5.48638068 5.48658085 5.48585374 5.48593428 5.48595645
 5.48600539 5.48383792 5.48439973 5.48518922 5.4927774  5.492

44 22
44 22
44 22
5.844528288642234 5.844528292891898 6.139552041023755
44 22
44 22
L_new 4096.0
[5.50752846 5.50809201 5.50706972 5.50775142 5.50544543 5.50769111
 5.50785509 5.50792813 5.5076498  5.50623738 5.5077506  5.50817774
 5.50747478 5.50767364 5.50787476 5.50714422 5.50722514 5.5072474
 5.50729659 5.50511884 5.5056833  5.50647654 5.51409977 5.51354643
 5.50927702 5.51013328 5.50829615 5.50887636 5.51412165 5.51540869
 5.51112118 5.50325934 5.50074396 5.50970629 5.50525452 5.51697093
 5.51524627 5.49940936 5.49905492 5.50046953 5.50489153 5.49915673
 5.50016892 5.49917928]
2
44 22
44 22
44 22
5.844528288642234 5.844528292891898 6.139552041023755
44 22
44 22
44 22
1834.7044413995027
0.1279586365761265 0.17175914825529662
44 22
[5.50886199 5.50942571 5.50840311 5.50908502 5.50677835 5.50902469
 5.50918872 5.50926178 5.50898337 5.50757053 5.5090842  5.50951146
 5.50880829 5.50900721 5.50920839 5.50847764 5.50855858 5.50858085
 5.50863005 5.50645166 5.50701629 5.50780976 5.5154351

 5.5215108  5.5205165 ]
2
44 22
44 22
44 22
5.844528220505582 5.844528224771965 6.1395366880750935
44 22
44 22
L_new 4096.0
[5.52890407 5.52947029 5.52844317 5.52912809 5.52681113 5.52906751
 5.52923226 5.52930564 5.52902599 5.52760688 5.52912726 5.52955642
 5.52885014 5.52904993 5.52925199 5.52851802 5.52859932 5.52862168
 5.5286711  5.52648303 5.52705016 5.52784718 5.53550557 5.5349496
 5.53066063 5.53152076 5.52967547 5.53025805 5.53552751 5.53682048
 5.53251322 5.52461536 5.5220882  5.53109174 5.52661956 5.53838996
 5.53665731 5.52074754 5.52039146 5.52181258 5.52625492 5.52049404
 5.5215108  5.5205165 ]
2
44 22
44 22
44 22
5.844528220505582 5.844528224771965 6.1395366880750935
44 22
44 22
44 22
1834.7044569275115
0.1279547853061854 0.1717591334217375
44 22
[5.53024282 5.53080921 5.52978178 5.53046691 5.52814926 5.53040631
 5.5305711  5.53064451 5.53036478 5.52894524 5.53046607 5.53089537
 5.53018887 5.53038872 5.53059084 5.52985666 5.52993798 5.52996035
 5.53000978 5.52782107 5.52

2
44 22
44 22
44 22
5.844528122089216 5.8445281263797515 6.139514519343816
44 22
44 22
44 22
1834.704479314501
0.12794922437474202 0.17175911199939564
44 22
[5.56112443 5.56169469 5.56066028 5.56135004 5.55901655 5.56128905
 5.56145496 5.56152887 5.56124723 5.55981802 5.5613492  5.56178142
 5.56107013 5.56127131 5.56147481 5.56073565 5.56081752 5.56084003
 5.56088982 5.55868619 5.55925734 5.56006003 5.56777143 5.5672115
 5.56289307 5.563759   5.56190142 5.56248757 5.56779346 5.56909537
 5.56475835 5.5568062  5.55426128 5.56332692 5.55882398 5.57067577
 5.56893108 5.5529115  5.55255295 5.55398386 5.55845686 5.55265668
 5.55368033 5.55267901]
2
44 22
44 22
44 22
5.844528117797628 5.844528122089216 6.1395135528326925
44 22
44 22
L_new 4096.0
[5.56112443 5.56169469 5.56066028 5.56135004 5.55901655 5.56128905
 5.56145496 5.56152887 5.56124723 5.55981802 5.5613492  5.56178142
 5.56107013 5.56127131 5.56147481 5.56073565 5.56081752 5.56084003
 5.56088982 5.55868619 5.55925734 5.56006003 5.567

KeyboardInterrupt: 

In [ ]:
np.dot(d_hat_new.T, np.ones(size)).A1 - l

In [ ]:
number_of_people = df.groupby('area_j')['live_i_work_j'].sum().sum()

In [ ]:
d_restored = d_hat_new * number_of_people

In [ ]:
np.sum((d_restored - corresponance_matrix) ** 2) / size ** 2

In [ ]:
np.sum((d_restored - corresponance_matrix) ** 2) / size ** 2